In [1]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3 # TypeError: dropout(): argument 'input' (position 1) must be Tensor, not str
!pip install torch

     |████████████████████████████████| 46.9 MB 44 kB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 344 kB 4.3 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595725 sha256=415a72dbf460ac1b32f1bd8a2f444192fa4ee83b65ec2eaf325682cd2ebd57f0
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
     |████████████████████████████████| 1.2 MB 2.9 MB/s 
     |████████████████████████████████| 754 kB 2.8 MB/s 
     |████████████████████████████████| 3.0 MB 34.9 MB/s 
     |████████████████████████████████| 895 kB 40.2 MB/s 


In [2]:
#깃허브에서 KoBERT 파일 로드
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-3_dgoz2n
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-3_dgoz2n
  Created wheel for kobert: filename=kobert-0.1.2-py3-none-any.whl size=12771 sha256=7ece184474ddf3c753a1ec88638e2ea9d7fdd678bc75f8b2f20e0f5ce8450ae3
  Stored in directory: /tmp/pip-ephem-wheel-cache-o1otztak/wheels/d3/68/ca/334747dfb038313b49cf71f84832a33372f3470d9ddfd051c0
Successfully built kobert


In [3]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm as tqdm, tqdm_notebook

In [4]:
#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

In [5]:
#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [6]:
# classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [7]:
#GPU 사용
device = torch.device("cuda:0")

In [8]:
#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]


In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
#불러올 파일의 경로를 filename 변수에 저장
filename = '/content/drive/My Drive/ESG/news_train_sample.csv'

In [11]:
# 학습용 데이터셋 불러오기
import pandas as pd
dataset = pd.read_csv(filename)
dataset

,title,originallink,link,description,pubDate,corp_name,E,S,G,value
0,"백악관 또 반도체회의, 삼성전자 불러놓고 ""내부 정보 내놔""",https://www.joongang.co.kr/article/25009320,https://news.naver.com/main/read.naver?mode=LS...,"이날 회의에는 삼성전자를 비롯해 TSMC, 인텔, 마이크로소프트(MS), 애플, 제...","Fri, 24 Sep 2021 10:11:00 +0900",삼성전자,0,0,0,0
1,"백악관, 또 반도체 대응 회의···삼성전자도 참석",https://www.sedaily.com/NewsView/22RL705FUP,https://news.naver.com/main/read.naver?mode=LS...,미국 백악관이 반도체 부족 문제를 해결하기 위해 삼성전자와 애플 등 주요 기업 관계...,"Fri, 24 Sep 2021 07:47:00 +0900",삼성전자,0,0,0,0
2,"NAVER, 규제 노이즈에도 투자포인트 `여전`…추가 하락시 `매수` -한국",http://www.edaily.co.kr/news/newspath.asp?news...,https://news.naver.com/main/read.naver?mode=LS...,한국투자증권은 15일 NAVER(035420)에 대해 장기적인 투자포인트의 본질은 ...,"Wed, 15 Sep 2021 08:04:00 +0900",네이버,0,0,0,0
3,'빅테크 규제'에 카카오·네이버를 바라보는 온도차,http://www.newsis.com/view/?id=NISX20210924_00...,https://news.naver.com/main/read.naver?mode=LS...,"기사내용 요약카카오 목표가 하향, 네이버는 유지 ""카카오 방향성 맞지만 성장세 둔화...","Fri, 24 Sep 2021 11:06:00 +0900",카카오,0,0,0,0
4,"삼성바이오로직스, 의약품 위탁개발 플랫폼 개발·공개",http://www.fnnews.com/news/202109221321036446,https://news.naver.com/main/read.naver?mode=LS...,"삼성바이오로직스가 세포주 개발부터 시험계획서(IND) 제출, 그리고 공정 특성확인(...","Wed, 22 Sep 2021 13:29:00 +0900",삼성바이오로직스,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
1920,"삼양식품, 미얀마 산업연수생에게 소송 취하 압력?",http://www.ohmynews.com/NWS_Web/view/at_pg.asp...,https://news.naver.com/main/read.naver?mode=LS...,[ 변호사와 이주노동자들이 (주)삼양식품을 근로기준법과 최저임금법을 위반한 혐의로 ...,"Thu, 12 Nov 2009 17:13:00 +0900",삼양식품,0,1,0,2
1921,"전인장 삼양식품 회장, 횡령으로 ‘징역 3년’ 구속…‘불닭볶음면 불매운동’...",http://www.ajunews.com/view/20190125145641625,http://www.ajunews.com/view/20190125145641625,"누리꾼들은 ""삼양식품 불닭볶음면 불매운동 해야겠다"" ""회장이라고 회삿돈을 마음대로 ...","Fri, 25 Jan 2019 15:10:00 +0900",삼양식품,0,0,1,2
1922,"[식품회사 도 넘은 대주주 배불리기 ①] 삼양식품, 오너회사와 불공정거래",NaN,https://news.naver.com/main/read.naver?mode=LS...,"같다""며 ""불공정거래행위가 의심된다""고 말했다. 이에 대해 최남석 삼양식품 홍보팀장...","Wed, 14 Mar 2012 15:03:00 +0900",삼양식품,0,0,1,2
1923,"김정수 삼양식품 총괄사장, 대표이사 대신 ESG위원장…왜?",http://www.sisajournal.com/news/articleView.ht...,https://news.naver.com/main/read.naver?mode=LS...,업계에서는 비리로 삼양식품의 도덕성에 막대한 타격을 입힌 김 총괄사장이 윤리경영을 ...,"Tue, 09 Mar 2021 14:22:00 +0900",삼양식품,0,0,1,0


In [12]:
value_data = dataset[['title','value']]

In [13]:
value_list = []
for q, label in zip(value_data['title'], value_data['value']):
    data = [q, str(int(label))]
    value_list.append(data)

value_list

[['백악관 또 반도체회의, 삼성전자 불러놓고 "내부 정보 내놔"', '0'],
 ['백악관, 또 반도체 대응 회의···삼성전자도 참석', '0'],
 ['NAVER, 규제 노이즈에도 투자포인트 `여전`…추가 하락시 `매수` -한국', '0'],
 ["'빅테크 규제'에 카카오·네이버를 바라보는 온도차", '0'],
 ['삼성바이오로직스, 의약품 위탁개발 플랫폼 개발·공개', '0'],
 ['연일 하락 카카오 시총 5위 자리도 내줬다…LG화학, GM 배터리 재공급 소식에...', '0'],
 ['LG·SK 공격적 투자에 고민 깊어지는 삼성SDI', '0'],
 ["삼성SDI vs LG엔솔, '코인셀' 배터리 경쟁 달아오른다", '0'],
 ["배터리 '소부장' 둥지 튼 삼성SDI·LG화학 임원들", '0'],
 ["현대차, 3분기 '흑자 전환'…반도체 수급난에도 '미국·유럽' 수요는 호재", '0'],
 ['삼성·LG·현대차, 국제 디자인상 IDEA 2021 ‘싹쓸이’', '0'],
 ["셀트리온 '펜타곤 잭팟' 7000억원 코로나 진단키트 공급", '0'],
 ["7382억 '셀트리온 매직'…진단키트로 '바이 아메리칸' 뚫었다", '0'],
 ['셀트리온, 최대 7382억원 코로나19 진단키트 美에 공급', '0'],
 ['금호타이어, 기아 EV6 신차용 타이어 공급기념 최대 40% 할인', '0'],
 ['금호타이어, 기아 EV6 신차용 타이어 공급 기념 프로모션', '0'],
 ['담원 기아-T1 결승전, 350만 명이 지켜보며 LCK 단일 경기 시청 기록 경신', '0'],
 ["현대모비스, 인천·울산에 1.3조 들여 '수소연료전지 공장' 신설", '1'],
 ['삼성물산 SSF샵 \'세사패 캠페인\' 진행…"온라인 강화 잰걸음"', '0'],
 ["삼성물산, 집코노미 박람회에 브랜드 '래미안 리뉴얼' 소개", '0'],
 ['삼성물산 패션 SSF샵, \'세사패 캠페인\'…"온라인 강화"', '0'],
 ['LG전자 디오스 김치톡톡 오브제컬렉션 신제품 출

In [14]:
# Train / Test set 분리
from sklearn.model_selection import train_test_split

v_train, v_test = train_test_split(value_list, test_size=0.2, random_state=42)

print("v_train shape is:", len(v_train))
print("v_test shape is:", len(v_test))

v_train shape is: 1540
v_test shape is: 385


In [15]:
# 기본 Bert tokenizer 사용
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model


In [16]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [17]:
# Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 9
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [18]:
v_data_train = BERTDataset(v_train, 0, 1, tok, max_len, True, False)
v_data_test = BERTDataset(v_test, 0, 1, tok, max_len, True, False)

In [19]:
# pytorch용 DataLoader 사용
v_train_dataloader = torch.utils.data.DataLoader(v_data_train, batch_size=batch_size, num_workers=2)
v_test_dataloader = torch.utils.data.DataLoader(v_data_test, batch_size=batch_size, num_workers=2)

In [20]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes = 3,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [21]:
v_model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)

In [22]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
v_optimizer_grouped_parameters = [
    {'params': [p for n, p in v_model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in v_model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [23]:
# 옵티마이저 선언
v_optimizer = AdamW(v_optimizer_grouped_parameters, lr=learning_rate)
v_loss_fn = nn.CrossEntropyLoss()

In [24]:
v_t_total = len(v_train_dataloader) * num_epochs * 2
warmup_step = int(v_t_total * warmup_ratio)

In [25]:
v_scheduler = get_cosine_schedule_with_warmup(v_optimizer, num_warmup_steps=warmup_step, num_training_steps=v_t_total)

In [26]:
# 학습 평가 지표인 accuracy 계산 -> 얼마나 타겟값을 많이 맞추었는가
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [27]:
# 모델 학습 시작
for v in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    # 학습모드
    v_model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(v_train_dataloader)):
        v_optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        v_out = v_model(token_ids, valid_length, segment_ids)
        loss = v_loss_fn(v_out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(v_model.parameters(), max_grad_norm) # gradient clipping
        v_optimizer.step()
        v_scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(v_out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(v+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(v+1, train_acc / (batch_id+1)))
    # 평가모드
    v_model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(v_test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        v_out = v_model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(v_out, label)
    print("epoch {} test acc {}".format(v+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  import sys


  0%|          | 0/25 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 1.105549693107605 train acc 0.40625
epoch 1 train acc 0.548125


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/7 [00:00<?, ?it/s]

...

epoch 8 test acc 0.921875


  0%|          | 0/25 [00:00<?, ?it/s]

epoch 9 batch id 1 loss 0.01286198478192091 train acc 1.0
epoch 9 train acc 0.98875


  0%|          | 0/7 [00:00<?, ?it/s]

epoch 9 test acc 0.9285714285714286


In [28]:
commnetslist = [] # 텍스트 데이터를 담을 리스트
v_emo_list = [] # 감성 값을 담을 리스트
for c in v_test: # 모든 댓글
  commnetslist.append( [c, 5] ) # [댓글, 임의의 양의 정수값] 설정

pdData = pd.DataFrame( v_test, columns = [['title', 'value']] )
pdData = pdData.values

test_set = BERTDataset(pdData, 0, 1, tok, max_len, True, False)
test_input = torch.utils.data.DataLoader(test_set, batch_size=batch_size, num_workers=5)

v_model.eval()

for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_input):
    token_ids = token_ids.long().to(device)
    segment_ids = segment_ids.long().to(device)

    valid_length= valid_length
    label = label.long().to(device)

    out = v_model(token_ids, valid_length, segment_ids)


    for i in out:
        logits=i
        logits = logits.detach().cpu().numpy()

        if np.argmax(logits) == 0:
            v_emo_list.append(0)
        elif np.argmax(logits) == 1:
            v_emo_list.append(1)
        elif np.argmax(logits) == 2:
            v_emo_list.append(2)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [29]:
v_checklist = []
for i in range(len(v_test)):
  v_checklist.append(int(v_test[i][1]))

In [30]:
# 테스트 결과 확인
confusion_matrix(v_emo_list, v_checklist)
print(classification_report(v_emo_list, v_checklist))

              precision    recall  f1-score   support

           0       0.91      0.96      0.93       221
           1       0.95      0.82      0.88        93
           2       0.91      0.90      0.91        71

    accuracy                           0.92       385
   macro avg       0.92      0.89      0.91       385
weighted avg       0.92      0.92      0.92       385



In [31]:
# 모델 저장
torch.save(v_model.state_dict, './drive/MyDrive/ESG/v_model.pt')